# `ipython`
> Set of utility functions to be used in Jupyter and Jupyter Lab notebooks.


In [ ]:
#|default_exp ipython

In [ ]:
#| export
from __future__ import annotations
from functools import wraps
from IPython.core.getipython import get_ipython
from IPython.display import display, Markdown, display_markdown
from pathlib import Path
from typing import Any, Callable, Optional
from ecutilities.core import validate_path, validate_type, IsLocalMachine

import configparser
import numpy as np
import os
import pandas as pd
import subprocess
import warnings

In [ ]:
#| hide
from nbdev import show_doc, nbdev_export

# System and CLI

In [ ]:
#| export
def run_cli(cmd:str = 'ls -l'   # command to execute in the cli
           ):
    """Runs a cli command from jupyter notebook and print the shell output message
    
    Uses subprocess.run with passed command to run the cli command"""
    p = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True)
    print(str(p.stdout, 'utf-8'))

In [ ]:
run_cli('pwd')

/home/vtec/projects/ec-packages/ecutilities/nbs-dev



# Notebook setup

In [ ]:
#| export
def nb_setup(autoreload:bool = True,   # True to set autoreload in this notebook
             paths:list(Path) = None   # Paths to add to the path environment variable
            ):
    """Use in first cell of notebook to set autoreload, and paths"""
#   Add paths. Default is 'src' if it exists
    if paths is None:
        p = Path('../src').resolve().absolute()
        if p.is_dir():
            paths = [str(p)]
        else:
            paths=[]
    if paths:
        for p in paths:
            sys.path.insert(1, str(p))
        print(f"Added following paths: {','.join(paths)}")

#   Setup auto reload
    if autoreload:
        ipshell = get_ipython()
        ipshell.run_line_magic('load_ext',  'autoreload')
        ipshell.run_line_magic('autoreload', '2')
        print('Set autoreload mode')

In [ ]:
show_doc(nb_setup)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L32){target="_blank" style="float:right; font-size:smaller"}

### nb_setup



Use in first cell of notebook to set autoreload, and paths

In [ ]:
nb_setup()

Set autoreload mode


By default, `ipython.nb_setup()` 
- loads and set `autoreload`
- adds a path to a directory named `src` when it exists at the same level as where the notebook directory is located. It no such `src` directory exists, no path is added

`ipython.nb_setup` assumes the following file structure:

```
    project_directory
          | --- notebooks
          |        | --- current_nb.ipynb
          |        | --- ...
          |
          |--- src
          |     | --- scripts_to_import.py
          |     | --- ...
          |
          |--- data
          |     |
          |     | ...
```

For other file structure, specify paths as a `list` of `Path`

In [ ]:
#| export
def cloud_install_project_code(
    package_name:str # project package name, e.g. metagentools or git+https://github.com/repo.git@main
):
    """When nb is running in the cloud, pip install the project code package"""
    
    # test whether it runs on colab
    try:
        from google.colab import drive
        RUN_LOCALLY = False
        print('The notebook is running on colab')

    except ModuleNotFoundError:
        # not running on colab, testing is it runs on on a local machine
        RUN_LOCALLY = IsLocalMachine().is_local()
        
        if RUN_LOCALLY:
            print('The notebook is running locally, will not automatically install project code')
        else:
            print('The notebook is running on a cloud VM or the machine was not registered as local')

    if not RUN_LOCALLY:
        print(f'Installing project code {package_name}')
        cmd = f"pip install -U {package_name}"
        run_cli(cmd)
        print((f"{package_name} is installed."))
        
    return RUN_LOCALLY

When using colab or another cloud VM, project code must be installed every time from the Python Package Index (PyPI) or its GitHub repo.

When running locally, the project code should be pre-installed as part of the environment

In [ ]:
cloud_install_project_code(package_name='metagentools');

The notebook is running locally, will not automatically install project code


# Improve output cell formats

In [ ]:
#| export
def display_mds(
    *strings:str|tuple[str] # any number of strings with text in markdown format
):
    """Display one or several strings formatted in markdown format"""
    for string in strings:
        display_markdown(Markdown(data=string))

In [ ]:
show_doc(display_mds)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L85){target="_blank" style="float:right; font-size:smaller"}

### display_mds

>      display_mds (*strings:str|tuple[str])

Display one or several strings formatted in markdown format

In [ ]:
display_mds('**bold** and _italic_')

**bold** and _italic_

In [ ]:
display_mds('**bold** and _italic_',
            '- bullet',
            '- bullet',
            '> Note: this is a note'
)

**bold** and _italic_

- bullet

- bullet

> Note: this is a note

In [ ]:
#| export
def display_dfs(*dfs:pd.DataFrame       # any number of Pandas DataFrames
               ):
    """Display one or several `pd.DataFrame` in a single cell output"""
    for df in dfs:
        display(df)

In [ ]:
show_doc(display_dfs)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L93){target="_blank" style="float:right; font-size:smaller"}

### display_dfs

>      display_dfs (*dfs:pandas.core.frame.DataFrame)

Display one or several `pd.DataFrame` in a single cell output

In [ ]:
df1 = pd.DataFrame(data=np.random.normal(size=(10,5)))
df2 = pd.DataFrame(data=np.random.normal(size=(20,10)))

display_dfs(df1.head(3), df2.head(3))

,0,1,2,3,4
0,-0.813233,-0.701623,-0.274101,1.285042,-0.784862
1,1.053325,1.220413,0.072135,1.390026,-1.549951
2,2.420573,-0.250810,0.585279,0.646174,-1.360917


,0,1,2,3,4,5,6,7,8,9
0,0.341911,0.380127,0.051711,1.160899,-1.627953,-0.460142,0.712536,0.216631,-0.154492,0.492482
1,0.185234,0.840259,0.188197,-0.549580,-0.361178,0.973196,-1.693053,-0.125455,-1.013398,0.422213
2,-1.105901,0.186083,0.659266,-0.831638,-1.303234,-2.478721,-1.240304,-0.409095,0.429070,0.700964


In [ ]:
#| export
class pandas_nrows_ncols:
    """Context manager to specify a max nbr of rows and cols to apply to Series and DataFrame outputs"""
    def __init__(
        self, 
        nrows:int|None=None, # maximum number of rows to show within the context
        ncols:int|None=None, # maximum number of columns to show within the context
    ):
        self.nrows = nrows
        self.ncols = ncols
    
    def __enter__(self):
        self.max_rows = pd.options.display.max_rows
        self.max_cols = pd.options.display.max_columns
        pd.options.display.max_rows = self.nrows
        pd.options.display.max_columns = self.ncols
        return self.max_rows, self.max_cols

    def __exit__(self, exc_type, exc_value, exc_tb):
        pd.options.display.max_rows = self.max_rows
        pd.options.display.max_columns = self.max_cols

In [ ]:
show_doc(pandas_nrows_ncols)

---

### pandas_nrows_ncols

>      pandas_nrows_ncols (nrows:int|None=None, ncols:int|None=None)

context manager to specify a max nbr of rows and cols to apply to Series and DataFrame outputs

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| nrows | int \| None | None | maximum number of rows to show within the context |
| ncols | int \| None | None | maximum number of columns to show within the context |

With no context manager, the pandas object are displayed with a maximum of 60 rows and 20 columns.

In [ ]:
df = pd.DataFrame(np.random.randint(low=0, high=100, size=(3,50)))
display(df)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,22,5,94,69,77,12,74,48,87,95,...,6,0,72,46,57,49,92,2,95,57
1,33,45,74,8,71,23,98,70,47,36,...,78,98,13,93,25,33,47,13,62,10
2,6,32,35,46,84,81,33,16,54,28,...,74,30,49,67,5,98,86,70,13,61


Using the context manager, all rows and columns will be displayed

In [ ]:
with pandas_nrows_ncols():
    display(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,22,5,94,69,77,12,74,48,87,95,65,46,16,77,14,92,3,48,61,62,90,25,19,67,96,26,11,95,12,39,41,2,22,55,37,78,25,98,29,9,6,0,72,46,57,49,92,2,95,57
1,33,45,74,8,71,23,98,70,47,36,87,71,69,56,3,52,45,51,23,64,55,84,28,31,53,11,36,62,60,73,51,62,47,84,91,50,52,34,31,27,78,98,13,93,25,33,47,13,62,10
2,6,32,35,46,84,81,33,16,54,28,64,78,9,35,2,64,63,68,79,32,88,88,97,4,51,49,20,72,61,83,39,13,34,55,47,33,5,31,30,15,74,30,49,67,5,98,86,70,13,61


It is also possible to specifically define the number of rows and columns to display

In [ ]:
with pandas_nrows_ncols(nrows=2, ncols=4):
    display(df)

,0,1,...,48,49
0,22,5,...,95,57
...,...,...,...,...,...
2,6,32,...,13,61


> **Technical background**:
> 
> the context manager uses pandas's [`options API`](https://pandas.pydata.org/docs/user_guide/options.html)

In [ ]:
pd.options.display.max_rows, pd.options.display.max_columns

(60, 20)

In [ ]:
pd.get_option('display.max_rows'), pd.get_option('display.max_columns')

(60, 20)

In [ ]:
pd.describe_option('display.max_rows')

display.max_rows : int
    If max_rows is exceeded, switch to truncate view. Depending on
    `large_repr`, objects are either centrally truncated or printed as
    a summary view. 'None' value means unlimited.

    In case python/IPython is running in a terminal and `large_repr`
    equals 'truncate' this can be set to 0 and pandas will auto-detect
    the height of the terminal and print a truncated object which fits
    the screen height. The IPython notebook, IPython qtconsole, or
    IDLE do not run in a terminal and hence it is not possible to do
    correct auto-detection.
    [default: 60] [currently: 60]


In [ ]:
pd.options.display.max_rows = 10
pd.reset_option('display.max_rows')
pd.options.display.max_rows

60

In [ ]:
import warnings
warnings.warn('blabla', category=DeprecationWarning)

/tmp/ipykernel_1448/3565374201.py:2: DeprecationWarning: blabla
  warnings.warn('blabla', category=DeprecationWarning)


In [ ]:
#| export
def df_all_cols_and_rows(
    f:Callable,   # function to apply the decorator ti
)-> Callable:     # decorated function
    """decorator function forcing all rows and columns of `DataFrames` to be displayed in the wrapped function"""
    
    msg = 'Decorator is deprecated and will be removed soon. Use context manager `pandas_nrows_ncols` instead.'
    warnings.warn(msg, category=DeprecationWarning)
    
    @wraps(f)
    def wrapper(*args, **kwargs):
        max_rows = pd.options.display.max_rows
        max_cols = pd.options.display.max_columns
        pd.options.display.max_rows = None
        pd.options.display.max_columns = None
        f(*args, **kwargs)
        pd.options.display.max_rows = max_rows
        pd.options.display.max_columns = max_cols
    
    return wrapper

In [ ]:
show_doc(df_all_cols_and_rows)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L100){target="_blank" style="float:right; font-size:smaller"}

### df_all_cols_and_rows

>      df_all_cols_and_rows (f:Callable)

decorator function forcing all rows and columns of `DataFrames` to be displayed in the wrapped function

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| f | Callable | function to apply the decorator ti |
| **Returns** | **Callable** | **decorated function** |

Usage of the decorator

In [ ]:
@df_all_cols_and_rows
def show_df(df):
    display(df)

df = pd.DataFrame(np.random.randint(low=0, high=100, size=(3,50)))
show_df(df)

/tmp/ipykernel_1448/3954027064.py:8: DeprecationWarning: Decorator is deprecated and will be removed soon. Use context manager `pandas_nrows_ncols` instead.
  warnings.warn(msg, category=DeprecationWarning)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,34,97,47,98,11,15,66,12,7,90,2,20,63,84,95,58,12,57,22,88,53,75,12,58,90,65,3,61,91,20,49,83,51,22,34,58,57,33,85,30,37,23,53,65,51,8,39,89,69,82
1,50,91,67,89,53,29,64,28,22,0,2,42,67,44,87,30,25,56,80,28,20,70,66,38,49,72,90,71,58,14,97,30,44,99,73,82,36,6,36,16,62,61,72,23,48,76,27,90,46,82
2,53,73,58,21,32,77,46,45,24,29,16,99,42,15,55,79,74,48,89,10,96,69,96,9,97,95,20,53,83,77,93,28,15,79,86,95,47,0,51,33,7,28,81,58,33,12,12,31,34,59


In [ ]:
#| export
@df_all_cols_and_rows
def display_full_df(
    df:pd.DataFrame  # DataFrame to display
):
    """Display a `DataFrame` showing all rows and columns"""
#     if not isinstance(df, pd.DataFrame): raise TypeError('df must me a pandas DataFrame')
    validate_type(df, pd.DataFrame, raise_error=True)
    display(df)

In [ ]:
show_doc(display_full_df)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L118){target="_blank" style="float:right; font-size:smaller"}

### display_full_df

>      display_full_df (df:pandas.core.frame.DataFrame)

Display a `DataFrame` showing all rows and columns

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| df | pd.DataFrame | DataFrame to display |

In [ ]:
df = pd.DataFrame(np.random.randint(low=0, high=100, size=(3,50)))
df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,91,38,70,46,35,58,31,80,94,42,...,66,95,46,91,45,41,22,85,6,59
1,12,80,32,13,10,36,1,20,63,89,...,54,55,39,19,10,38,81,58,70,72
2,65,95,17,11,24,11,46,74,88,95,...,78,72,56,78,62,58,81,39,16,70


In [ ]:
display_full_df(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,91,38,70,46,35,58,31,80,94,42,12,56,90,81,30,24,88,68,52,4,94,85,73,14,77,42,69,1,92,73,65,5,4,23,31,10,81,8,61,37,66,95,46,91,45,41,22,85,6,59
1,12,80,32,13,10,36,1,20,63,89,14,62,48,81,68,23,52,66,31,29,98,63,80,35,15,22,99,13,74,29,90,87,17,34,13,66,76,13,18,39,54,55,39,19,10,38,81,58,70,72
2,65,95,17,11,24,11,46,74,88,95,61,19,12,48,72,93,45,4,85,17,3,51,98,33,75,16,33,76,52,18,48,12,97,88,32,86,3,30,16,51,78,72,56,78,62,58,81,39,16,70


In [ ]:
#| hide
nbdev_export()